In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
X = pd.read_csv("/content/X_features.csv", index_col=0, parse_dates=True)
y = pd.read_csv("/content/y_target.csv", index_col=0, parse_dates=True)

X.head(), y.head()

(               lag_1     lag_2     lag_5    lag_10  roll_mean_5  roll_std_5  \
 Date                                                                          
 2020-10-28  0.095767  0.047334 -0.007580 -0.013823     0.032992    0.047521   
 2020-10-29 -0.009174  0.095767  0.050858  0.059238     0.020969    0.049437   
 2020-10-30 -0.009259 -0.009174 -0.019823 -0.020388     0.013053    0.059705   
 2020-11-02 -0.059404 -0.009259  0.047334 -0.014523     0.011521    0.058695   
 2020-11-03  0.039676 -0.059404  0.095767 -0.031850    -0.008393    0.035126   
 
             roll_mean_20  roll_std_20   ewma_10   ewma_20  abs_return  \
 Date                                                                    
 2020-10-28      0.006644     0.040158  0.018098  0.012191    0.009174   
 2020-10-29      0.006392     0.040245  0.013049  0.009864    0.009259   
 2020-10-30      0.004815     0.042227 -0.000285  0.002446    0.059404   
 2020-11-02      0.007731     0.042534  0.007053  0.006386    0.0396

In [5]:
def walk_forward_ml(model, X, y, min_train_size=500):
    predictions = []
    actuals = []

    for t in range(min_train_size, len(X)):
        X_train = X.iloc[:t]
        y_train = y.iloc[:t]

        X_test = X.iloc[t:t+1]
        y_test = y.iloc[t]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)[0]

        predictions.append(y_pred)
        actuals.append(y_test)

    return np.array(predictions), np.array(actuals)


In [6]:
rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=6,
    min_samples_leaf=20,
    random_state=42,
    n_jobs=-1
)

rf_preds, rf_actuals = walk_forward_ml(rf, X, y)
rf_rmse = np.sqrt(mean_squared_error(rf_actuals, rf_preds))
rf_rmse

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usi

np.float64(0.041645749614906365)

In [7]:
  gbr = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)

gbr_preds, gbr_actuals = walk_forward_ml(gbr, X, y)
gbr_rmse = np.sqrt(mean_squared_error(gbr_actuals, gbr_preds))
gbr_rmse

/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a

np.float64(0.044120448798171015)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(
    n_estimators=80,
    learning_rate=0.1,
    max_depth=2,
    random_state=42
)

gbr_preds, gbr_actuals = walk_forward_ml(
    gbr,
    X.iloc[:700],
    y.iloc[:700],
    window=500
)

from sklearn.metrics import mean_squared_error
gbr_rmse = np.sqrt(mean_squared_error(gbr_actuals, gbr_preds))

print("Gradient Boosting RMSE:", gbr_rmse)

TypeError: walk_forward_ml() got an unexpected keyword argument 'window'

In [ ]:
def walk_forward_ml_rolling(model, X, y, window=500):
    preds, actuals = [], []

    for t in range(window, len(X)):
        X_train = X.iloc[t-window:t]
        y_train = y.iloc[t-window:t]

        X_test = X.iloc[t:t+1]
        y_test = y.iloc[t]

        model.fit(X_train, y_train)
        preds.append(model.predict(X_test)[0])
        actuals.append(y_test)

    return np.array(preds), np.array(actuals)


In [ ]:
gbr_preds, gbr_actuals = walk_forward_ml_rolling(
    gbr,
    X.iloc[:700],
    y.iloc[:700],
    window=500
)


/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a

In [8]:
from sklearn.metrics import mean_squared_error
import numpy as np

gbr_rmse = np.sqrt(mean_squared_error(gbr_actuals, gbr_preds))
gbr_rmse

np.float64(0.044120448798171015)

In [9]:
rf_rmse

np.float64(0.041645749614906365)

In [10]:
print("Random Forest RMSE:", rf_rmse)
print("Gradient Boosting RMSE:", gbr_rmse)

Random Forest RMSE: 0.041645749614906365
Gradient Boosting RMSE: 0.044120448798171015


In [12]:
print(type(rf_actuals), np.array(rf_actuals).shape)
print(type(rf_preds),   np.array(rf_preds).shape)

<class 'numpy.ndarray'> (791, 1)
<class 'numpy.ndarray'> (791,)


In [13]:
rf_actuals_1d = np.asarray(rf_actuals).ravel()
rf_preds_1d   = np.asarray(rf_preds).ravel()


In [14]:
pd.DataFrame({
    "actual": rf_actuals_1d,
    "ml_pred": rf_preds_1d
}).to_csv("/content/ml_wfo_predictions.csv", index=False)


In [15]:
import joblib

joblib.dump(rf, "/content/rf_model.pkl")


['/content/rf_model.pkl']